# Imports and Installs

In [1]:
# install latest version of polyglot language detection library for stretch
# !pip install -U git+https://github.com/aboSamoor/polyglot.git@master

!pip install plotly

In [2]:
import pandas as pd
import numpy as np 
#import pandas_profiling 
from sklearn import preprocessing # for category encoder
# from polyglot.detect import Detector # for language detection stretch
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
# much more efficient for larger files like Nearest Neighbors which the model
import joblib
import plotly.express as px


In [ ]:
def create_web_chart(df):
    music_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']
    
    fig = go.Figure()

    for idx, row in df.iterrows():
        # vals = list(df.iloc[idx][music_features].values)
        row_df = pd.DataFrame(df.loc[idx][music_features].T).reset_index().rename(columns={idx:'values', 'index':'feature'})
        fig.add_trace(go.Scatterpolar(
                                    r=row_df['values'],
                                    theta=row_df['feature'],
                                    fill='toself',
                                    name= f'{row.artist_name} - {row.track_name}'
                    ))
    # fig.show()
    return fig

fig = create_web_chart(df.iloc[0:5])
fig.show()



In [ ]:
first_song_values = list(df.iloc[0][music_features].values)
second_song_values = list(df.iloc[1][music_features].values)

fig = px.bar(first_song_df, x='feature', y="values", color='feature')
fig.show()

In [ ]:
fig.add_trace(go.Scatterpolar(
      r=second_song_df['values'],
      theta=second_song_df['feature'],
      fill='toself',
      name='Second Song'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True
)

fig.show()

# EDA & PROFILE REPORT

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv')
# commented out profile report because it takes forever to run
# pandas_profiling.ProfileReport(df)

# DATA ENGINEERING CATEGORY ENCODING OF OBJECT FEATURES
note: might need to create another column of encoded languages
resources below: https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/


In [5]:
# # test of polyglot detector

# arabic_text = u"""
# أفاد مصدر امني في قيادة عمليات صلاح الدين في العراق بأن " القوات الامنية تتوقف لليوم
# الثالث على التوالي عن التقدم الى داخل مدينة تكريت بسبب
# انتشار قناصي التنظيم الذي يطلق على نفسه اسم "الدولة الاسلامية" والعبوات الناسفة
# والمنازل المفخخة والانتحاريين، فضلا عن ان القوات الامنية تنتظر وصول تعزيزات اضافية ".
# """

# detector = Detector(arabic_text)
# print(detector.language)

"""
   TODO: figure out UTF-8 ERROR or just do try catch exception and return 
   unknown or something
   below code does not work
"""

# df.track_name = df.track_name.astype(str)
# df['poly_obj'] = df.track_name.apply(lambda x: Detector(x, quiet=True))
# df['Track-lang'] = df['poly_obj'].apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
# df['Track-LangConfidence'] = df['poly_obj'].apply( lambda x: x.language.confidence)


'\n   TODO: figure out UTF-8 ERROR or just do try catch exception and return \n   unknown or something\n   below code does not work\n'

In [8]:
df = df.dropna() # drop null values

time_sig_encoding = { '0/4' : 0, '1/4' : 1, 
                     '3/4' : 3, '4/4' : 4,
                     '5/4' : 5}

key_encoding = { 'A' : 0, 'A#' : 1, 'B' : 2,
                'C' : 3,  'C#' : 4,  'D' : 5,
                'D#' : 6, 'E' : 7, 'F' : 8,
                'F#' : 9, 'G' : 10, 'G#' : 11 }

mode_encoding = { 'Major':0, 'Minor':1}      

df['key'] = df['key'].map(key_encoding)
df['time_signature'] = df['time_signature'].map(time_sig_encoding)
df['mode'] = df['mode'].map(mode_encoding)

# helper function to one hot encode genre

def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

df = encode_and_bind(df, 'genre')

# df = df.dropna() # drop null values again not sure why it created null values

In [10]:
df.sort_values('popularity', ascending=False)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,genre_Pop,genre_R&B,genre_Rap,genre_Reggae,genre_Reggaeton,genre_Rock,genre_Ska,genre_Soul,genre_Soundtrack,genre_World
9027,Dance,Ariana Grande,7 rings,14msK75pk3pA33pzPVNtBF,100,0.578000,0.725,178640,0.321,0.000000,...,0,0,0,0,0,0,0,0,0,0
107804,Pop,Ariana Grande,7 rings,14msK75pk3pA33pzPVNtBF,100,0.578000,0.725,178640,0.321,0.000000,...,1,0,0,0,0,0,0,0,0,0
86951,Rap,Post Malone,Wow.,6MWtB6iiXyIwun0YzU6DFP,99,0.163000,0.833,149520,0.539,0.000002,...,0,0,1,0,0,0,0,0,0,0
107803,Pop,Post Malone,Wow.,6MWtB6iiXyIwun0YzU6DFP,99,0.163000,0.833,149520,0.539,0.000002,...,1,0,0,0,0,0,0,0,0,0
107802,Pop,Ariana Grande,"break up with your girlfriend, i'm bored",4kV4N9D1iKVxx1KLvtTpjS,99,0.042100,0.726,190440,0.554,0.000000,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195435,Movie,Sally Dworsky,Inside Out,5d5BLVzCOxhgN02r2XqPWw,0,0.000305,0.456,241400,0.804,0.000018,...,0,0,0,0,0,0,0,0,0,0
195434,Movie,Mike Douglas,September Song,5VSpKjPu6bud6KHyyWQrJ1,0,0.839000,0.331,282720,0.221,0.000189,...,0,0,0,0,0,0,0,0,0,0
195433,Movie,Keith David,The Christmas Story,5JdfRifr7rjLNy9kegVu5z,0,0.653000,0.604,207040,0.365,0.000097,...,0,0,0,0,0,0,0,0,0,0
195432,Movie,Charlton Heston,Chorus: Come And Go With Me,5IF0TxLi4Jzo0qEdxhZbUy,0,0.972000,0.482,83667,0.312,0.000000,...,0,0,0,0,0,0,0,0,0,0


# MODELING: Nearest Neighbors
resources: https://scikit-learn.org/stable/modules/neighbors.html

In [25]:
neigh = NearestNeighbors(n_neighbors=11)

In [26]:
# to remove the transformed columns from model 
# remove = ['key', 'mode','time_signature']
# features = [i for i in list(df.columns[4:]) if i not in remove]

features = list(df.columns[4:])

# target = 'track_id'

In [27]:
X = df[features].values
# y = df[target]

X.shape # y.shape

(232725, 41)

In [28]:
%time
neigh.fit(X) # NN doesn't need to fit Y

Wall time: 0 ns


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                 radius=1.0)

# Model Test on Song Example

In [29]:
# highest ranking popularity song from Ariana Grande
test_song = X[9027] 

df.iloc[9027]

genre                                      Dance
artist_name                        Ariana Grande
track_name                               7 rings
track_id                  14msK75pk3pA33pzPVNtBF
popularity                                   100
acousticness                               0.578
danceability                               0.725
duration_ms                               178640
energy                                     0.321
instrumentalness                               0
key                                            4
liveness                                  0.0884
loudness                                 -10.744
mode                                           1
speechiness                                0.323
tempo                                     70.142
time_signature                                 4
valence                                    0.319
genre_A Capella                                0
genre_Alternative                              0
genre_Anime         

In [30]:
distance, neighbors = neigh.kneighbors(np.array([test_song]))

distance

array([[ 0.        ,  1.41421356, 27.00710509, 27.04410703, 45.58833874,
        45.87260502, 45.87260502, 50.40417417, 55.13866387, 55.32089875,
        56.71090497]])

In [31]:
neighbors

array([[  9027, 107804,  13706, 108810, 139423,  95537,  42041,  19588,
         11836, 227343, 215825]], dtype=int64)

In [38]:
song_list = []
for idx in neighbors[0][2:]: # this way excludes itself
    row = df.iloc[idx]
    print(f'Artist: {row.artist_name} - Track: {row.track_name}')
    song_list.append((row.artist_name, row.track_name))
song_list

Artist: Ariana Grande - Track: 7 rings (feat. 2 Chainz) - Remix
Artist: Ariana Grande - Track: 7 rings (feat. 2 Chainz) - Remix
Artist: Fanny Lu - Track: Romper el Hielo
Artist: Sharon Van Etten - Track: The End of the World
Artist: Sharon Van Etten - Track: The End of the World
Artist: Mike Perry - Track: Body to Body
Artist: Mon Laferte - Track: El Beso
Artist: Madi Sipes & The Painted Blue - Track: Space
Artist: Johnny Cash - Track: We'll Meet Again


[('Ariana Grande', '7 rings (feat. 2 Chainz) - Remix'),
 ('Ariana Grande', '7 rings (feat. 2 Chainz) - Remix'),
 ('Fanny Lu', 'Romper el Hielo'),
 ('Sharon Van Etten', 'The End of the World'),
 ('Sharon Van Etten', 'The End of the World'),
 ('Mike Perry', 'Body to Body'),
 ('Mon Laferte', 'El Beso'),
 ('Madi Sipes & The Painted Blue', 'Space'),
 ('Johnny Cash', "We'll Meet Again")]

# Helper Function to Return 10 Nearest Neighbors

In [70]:
"""
function for what was done manually above
generalized for other dataframes instead of hard coded therefore:
takes in a dataframe, a feature set array, and a song ID
and returns a list of tuples of artist name and track names
"""
from typing import List, Tuple

def closest_ten(df: pd.DataFrame, X_array: np.ndarray ,song_id: int) -> List[Tuple] :
    song = df.iloc[song_id]
    X_song = X[song_id]
    _, neighbors = neigh.kneighbors(np.array([X_song]))
    song_list = []
    for idx in neighbors[0][2:]: 
        row = df.iloc[idx]
       # print(f'Artist: {row.artist_name} - Track: {row.track_name}')
        song_list.append((row.artist_name, row.track_name))
    return song_list

In [71]:
# example of function call 
closest_ten(df, X, 9027)

[('Ariana Grande', '7 rings (feat. 2 Chainz) - Remix'),
 ('Ariana Grande', '7 rings (feat. 2 Chainz) - Remix'),
 ('Fanny Lu', 'Romper el Hielo'),
 ('Sharon Van Etten', 'The End of the World'),
 ('Sharon Van Etten', 'The End of the World'),
 ('Mike Perry', 'Body to Body'),
 ('Mon Laferte', 'El Beso'),
 ('Madi Sipes & The Painted Blue', 'Space'),
 ('Johnny Cash', "We'll Meet Again")]

# PCA 

In [77]:
df_copy = df[features].copy()

In [84]:
from numpy import array, mean, std, cov
from numpy.linalg import eig
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [85]:
means = df_copy.mean()
print("\n Means:", means)
scaler = StandardScaler()


 Means: popularity                    41.127502
acousticness                   0.368560
danceability                   0.554364
duration_ms               235122.339306
energy                         0.570958
instrumentalness               0.148301
key                            5.344467
liveness                       0.215009
loudness                      -9.569885
mode                           0.347969
speechiness                    0.120765
tempo                        117.666585
time_signature                 3.885148
valence                        0.454917
genre_A Capella                0.000511
genre_Alternative              0.039802
genre_Anime                    0.038397
genre_Blues                    0.038771
genre_Children's Music         0.023216
genre_Children’s Music         0.040189
genre_Classical                0.039772
genre_Comedy                   0.041598
genre_Country                  0.037228
genre_Dance                    0.037387
genre_Electronic               

In [90]:
pca_X = scaler.fit_transform(df_copy.values)
pca = PCA(2)
df_pca = pca.fit(pca_X)

In [93]:
%time
pca_neigh = NearestNeighbors(n_neighbors=11)
pca_neigh.fit(pca_X) # NN doesn't need to fit Y

Wall time: 0 ns


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                 radius=1.0)

# Export Model with Joblib

In [20]:
filename = 'NearestNeighborGenres.sav'

In [21]:
joblib.dump(neigh, filename)

['NearestNeighborGenres.sav']

In [95]:
pca_filename = 'PCANearestNeighborGenres.sav'
joblib.dump(pca_neigh, pca_filename)

['PCANearestNeighborGenres.sav']

In [22]:
# one hot encoding, rolled back
# def encode_and_bind(original_dataframe, feature_to_encode):
#     dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
#     res = pd.concat([original_dataframe, dummies], axis=1)
#     return(res)

# df = encode_and_bind(df, 'key')
# df = encode_and_bind(df, 'mode')
# df = encode_and_bind(df, 'time_signature')

# Export Model with Pickle

In [24]:
import pickle

model = neigh

filename = 'filename.pkl'

with open(filename, 'wb') as f:
    pickle.dump(model, f)
#then send this pkl file to DE team

#DE team:
#Make sure to save the pkl file in the same folder as the app

#To read it into the app:

# with open(filename, 'rb') as f:
#     model = pickle.load(f)